# Basic Ontology Interface Demo

this demonstrates the use of the BasicOntologyInterface which provides simplified access to local or remote
ontologies.

We demonstrate the use of different backends, but in practice you will likely only use one depending on your use case.

- pronto or sqlite for working with ontologies which you have a local copy of, and can trade startup time for generally faster operations
- ubergraph or ontobee or ols or bioportal for an ontology that has been loaded into a remote server

## Loading from obo files using Pronto

In [1]:
from obolib.implementations.pronto.pronto_implementation import ProntoImplementation
from obolib.resource import OntologyResource

### Local files

First we demonstrate loading from a file on the filesystem

In [2]:
resource = OntologyResource(slug='go-nucleus.obo', directory='input', local=True)
oi = ProntoImplementation.create(resource)

In [3]:
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

  rdfs:subClassOf ! subClassOf
    GO:0005773 ! vacuole
    GO:0043231 ! intracellular membrane-bounded organelle
  part_of ! part_of
    GO:0005737 ! cytoplasm


### Remote (downloading from OBO)

Next we use pronto's load from obo library feature

In [4]:
oi = ProntoImplementation.create(OntologyResource(local=False, slug='go.obo'))

In [5]:
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

  rdfs:subClassOf ! subClassOf
    GO:0005773 ! vacuole
    GO:0043231 ! intracellular membrane-bounded organelle
  part_of ! part of
    GO:0005737 ! cytoplasm


## SQL Database access

We can load from a SQL Database following Semantic SQL patterns

In [6]:
from obolib.implementations.sqldb.sql_implementation import SqlImplementation


In [7]:
oi = SqlImplementation.create(OntologyResource(slug=f'sqlite:///input/go-nucleus.db'))

CREATING: sqlite:///input/go-nucleus.db


In [8]:
## TODO
#rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
#for k, v in rels.items():
#    print(f'{k} = {v}')

In [9]:
for curie in oi.basic_search('intracellular'):
    print(f' MATCH: {curie} ! {oi.get_label_by_curie(curie)} ')

%intracellular%
PREDS = ['rdfs:label', 'oio:hasRelatedSynonym', 'oio:hasNarrowSynonym', 'oio:hasExactSynonym', 'oio:hasBroadSynonym']
 MATCH: GO:0005622 ! intracellular anatomical structure 
 MATCH: GO:0043231 ! intracellular membrane-bounded organelle 
 MATCH: GO:0031090 ! organelle membrane 
 MATCH: GO:0043229 ! intracellular organelle 


In [10]:
oi.interfaces_implemented()

[obolib.interfaces.basic_ontology_interface.BasicOntologyInterface]

## Loading from OWL ontologies using owlfun

TODO

## Wrapping remote ontology portals

### OLS

TODO

### BioPortal

TODO

## Wrapping SPARQL Endpoints

### Ubergraph

In [11]:
from obolib.implementations.ubergraph.ubergraph_implementation import UbergraphImplementation
oi = UbergraphImplementation.create()

URL = https://ubergraph.apps.renci.org/sparql


In [12]:
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

ERROR:root:Multiple labels for BFO:0000050 = ['part of', 'part_of', 'part of']


  BFO:0000050 ! part of
    GO:0005737 ! cytoplasm
  caro#part:of ! None
    GO:0005737 ! cytoplasm
  rdfs:subClassOf ! None
    GO:0043231 ! intracellular membrane-bounded organelle


### notes

Notice some of the differences with some of the other mechanisms:

 - ubergraph includes multiple ontologies, one of which 'injects' an legacy caro#part_of relationship
 - similarly there are different injected labels for the part-of relation
 - note also that the ubergraph implementation uses the actual predicate CURIE, currently pronto uses the shortname
 
This also involves multiple iterative calls to the API which is inefficient.

In future there will be an interface for 'bigger' operations that can be implemented more efficiently



### Ontobee

TODO

In [13]:
from obolib.implementations.ontobee.ontobee_implementation import OntobeeImplementation
oi = OntobeeImplementation.create()

URL = http://sparql.hegroup.org/sparql


In [14]:
parents = oi.get_parents_by_curie('GO:0005773')

In [15]:
#for parent in parents:
#    print(f'    {parent} ! {oi.get_label_by_curie(parent)}')